## Don't forget to change the paths

In [1]:
from threading import Thread,Lock
import cv2
from cv2 import imwrite
import time
from time import monotonic as timer
import os
import cv2
from ExecuteGesture import scale,swipe,minimize
from pynput.keyboard import Key
import numpy as np
import queue
import threading
import random
import imutils
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from tensorflow.keras.models import load_model
#import tensorflow.compat.v1 as tf
import tensorflow as tf

In [2]:
!pip list | grep tensorflow

tensorflow                             2.0.0      
tensorflow-estimator                   2.0.1      
tensorflow-object-detection-api        0.1.1      


In [3]:
#Modelo de deteção de mão
from object_detection.utils import ops as utils_ops
from object_detection.utils import visualization_utils as vis_util

# Load do modelo do tensorflow, que faz a deteção da mão

In [4]:
path = "/Users/guilhermeviveiros/Desktop/models/research/object_detection"
MODEL_NAME = 'hand_graph'
PATH_TO_CKPT = path + '/' + MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join(path+'/training', 'labelmap.pbtxt')
NUM_CLASSES = 1

In [5]:
detection_graph = tf.compat.v1.get_default_graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.compat.v2.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [6]:
from object_detection.utils import label_map_util
label_map_util.tf = tf.compat.v1
tf.gfile = tf.io.gfile

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Contador de tempo

In [7]:
from datetime import datetime

class CountsPerSec:
    """
    Class that tracks the number of occurrences ("counts") of an
    arbitrary event and returns the frequency in occurrences
    (counts) per second. The caller must increment the count.
    """

    def __init__(self):
        self._start_time = None
        self._num_occurrences = 0

    def start(self):
        self._start_time = datetime.now()
        return self

    def increment(self):
        self._num_occurrences += 1

    def countsPerSec(self):
        
        elapsed_time = (datetime.now() - self._start_time).total_seconds()
        return self._num_occurrences / elapsed_time

In [8]:
import argparse
import cv2


def putIterationsPerSec(frame, iterations_per_sec):
    """
    Add iterations per second text to lower-left corner of a frame.
    """

    cv2.putText(frame, "{:.0f} iterations/sec".format(iterations_per_sec),
        (10, 450), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255))
    return frame

In [9]:
#Vou escolher random entre as frames que tenho num segundo até ter 12 frames
#Exemplo -> Se obtiver 36 frames por segundo retiro apenas 12 aleatoriamente
#        -> Este processo é repetido 3 vezes para os 3 segundos
#        -> Passo estes 3 gestos de 36 frames à rede e fico com a moda dos resultados

class Sub_Distributed_frames():
    import random
    
    def __init__(self,frames):
        self.random1 = self.random(2,frames)
        self.random2 = self.random(10,frames)
        self.random3 = self.random(7,frames)
        self.l = len(frames)
        
    def getrandom(self):
        return self.random1
    
    def random(self,seed,frames):
        random.seed(seed)
        final_idx = []
        
        
        if(len(frames)) < 12:
            return list(cv2.resize(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), (75,100))/255 for frame in frames)
        
        else:
            
            total_divisions = len(frames) // 12
        
            #vou de retirar 12 frames de todos os blocos de frames que o total_divisions me devolve
            for i in range(0,12):
            
                #lista currente de indices
                list_idx = [tmp for tmp in range(total_divisions*i,total_divisions*(i+1))]
                #escolho 1 indice aleatório
                idx = random.choices(list_idx, k=1)[0]
                #acrescento-o à lista
                final_idx.append(idx)
        
    
            return list(cv2.resize(cv2.cvtColor(frames[i], cv2.COLOR_BGR2RGB), (75,100))/255 for i in final_idx )

In [10]:
class DistributedFrames():
    #frames ficam distribuidas em 3 listas, correspondetes a cada segundo
    #cada lista é utilizada por uma thread para processamento
    #utilizo listas, pois como demonstrado em cima são mais rápidas
    
    def __init__(self):
        
        from tensorflow.keras.models import load_model
        self.model = load_model('/Users/guilhermeviveiros/Desktop/best_model_10_sgd_nesterov_800k.h5')
        
        self.list1 = [] #lista1
        self.list2 = [] #lista2
        self.list3 = [] #lista3
        
        self.current_list = 1
        
        self.e1 = threading.Event() #evento para a thread1
        self.e2 = threading.Event() #evento para a thread2
        self.e3 = threading.Event() #evento para a thread3
        
        self.t1 = threading.Thread(target=self.thread_function, args=(1,self.e1), daemon=True) #thread1
        self.t2 = threading.Thread(target=self.thread_function, args=(2,self.e2), daemon=True) #thread2 
        self.t3 = threading.Thread(target=self.thread_function, args=(3,self.e3), daemon=True) #thread3
        
        self.t1.start()
        self.t2.start()
        self.t3.start()
        
        self.stopped = False
        self.make_copies = False
        
        self.copies = {}
        self.await_futture_list = True
        
    
    def list_prox(self):
        
        cl = self.current_list
        
        #caso esteja na lista1, thread1 processa e siga
        if(cl == 1):
            self.set_event(1)
            while(len(self.list1)>12):
                time.sleep(0.001)
                
            self.current_list +=1
            
        #caso esteja na lista2, thread2 processa e siga
        elif(cl == 2):
        
            self.set_event(2)
            while(len(self.list2)>12):
                time.sleep(0.001)
            
            self.current_list +=1
        
        #caso esteja na lista3, thread3 processa, dá pull da thread mais antiga e volta à thread 1
        elif(cl == 3):
            self.set_event(3)
            
            
            while(len(self.list3)>12):
                time.sleep(0.001)
                
            #print("Chegamos ao fim de 3 segundos")
            #print("Lista 1 com " + str(len(self.list1)))
            #print("Lista 2 com " + str(len(self.list2)))
            #print("Lista 3 com " + str(len(self.list3)))
            
            #Caso tenha previsto a mão, faz cópias para a memória das frames
            if(self.make_copies == True):
                #na primeira iteração guarda o passado e o presente
                if(self.await_future_list == True):
                    self.copies["list1"] = self.list2
                    self.copies["list2"] = self.list3
                    self.await_future_list = False
                #na segunda iteração guarda o "futuro"
                else:
                    self.copies["list3"] = self.list3
                    self.make_copies = False
            
            else:
                self.list1 = self.list2
                self.list2 = self.list3
                self.list3 = []
                
            
            
    def stop(self):
        self.stopped = True
    
    def append(self,frame):
        
        l = self.current_list
        
        if(l == 1):
            self.list1.append(frame)
        elif(l == 2):
            self.list2.append(frame)
        elif(l == 3):
            self.list3.append(frame)
    
    def set_event(self,id_thread):
        if(id_thread == 1):
            self.e1.set()
        elif(id_thread == 2):
            self.e2.set()
        elif(id_thread == 3):
            self.e3.set()
               
    #thread function
    #**
    #id_thread - representa o id_thread
    #e - representa o evento da thread
    #**
    def thread_function(self,id_thread,e):
        #enquanto não estiver pronto para executar espera    
        while not e.isSet():
            e.wait()
            if(id_thread == 1):
                self.list1 = Sub_Distributed_frames(self.list1).random1
            
            elif(id_thread == 2):
                self.list2 = Sub_Distributed_frames(self.list2).random1
                
            elif(id_thread == 3):
                self.list3 = Sub_Distributed_frames(self.list3).random1        
                
                #volta a bloquear a flag apenas se for a última thread, enquanto a thread main não der e.set fica bloequado
                if(self.stopped == False):
                    e.clear()
            
            #apenas bloqueio na última porque as outras qd acabam de fazer a operação uma vez, morrem
        
            
            print("The list " + str(id_thread) + " is concluded")
            
    def predict_model():
        self.make_copies = True
        
        while(self.make_copies == True):
            time.sleep(0.3)
        
        for i in self.copies:
            print(i)
        
        
        
        
        
#df = DistributedFrames()
#df

In [11]:
class TimeMarker():
    
    def __init__(self):
        self.stopped = False
        
    def start(self):
        Thread(target=self.set_marker, args=()).start()
        
    def set_marker(self):
        global lock
        global distributed_frames
        while not self.stopped:
            
            endtime = timer() + 1
            while timer() <= endtime:
                pass
            
            lock.acquire()
            distributed_frames.list_prox()
            lock.release()
        
    def stop(self):
        self.stopped = True

In [12]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

class is_hand():
    
    def __init__(self):
        
        self.e = threading.Event() #evento para a thread1
        self.t1 = threading.Thread(target=self.process_frame, args=(self.e,)) #thread1
        self.is_hand = False
        self.frame = None
        self.processed = False
        
        self.t1.start()
    
    def check_hand(self,frame):
                
        self.frame = frame
        self.e.set()
        
        while(self.processed == False):
            time.sleep(0.01)
        
        self.processed = False
        return self.is_hand
        
    
    def process_frame(self,e):
        
        while not e.isSet():
            
            e.wait()
            
            with detection_graph.as_default():
                
                with tf.Session(graph=detection_graph) as sess:
                                        
                    #frame = imutils.resize(self.frame, width=600)
                    frame = cv2.cvtColor(self.frame, cv2.COLOR_BGR2RGB)
                

                    image_np = frame
                    image_np_expanded = np.expand_dims(image_np,axis=0)
                    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
                    boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
                    scores = detection_graph.get_tensor_by_name('detection_scores:0')
                    classes = detection_graph.get_tensor_by_name('detection_classes:0')
                    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
                              
                    (boxes, scores, classes, num_detections) = sess.run(
                    [boxes, scores, classes, num_detections],
                    feed_dict={image_tensor: image_np_expanded})
                                        
                    condition = scores[0][0]*100                    
                    
                    if(condition >= 90): 
                        self.is_hand = True
                    
                    self.processed = True
                        
            e.clear()
                        
                        

Instructions for updating:
non-resource variables are not supported in the long term


In [13]:
class VideoGet():
    """
    Class that continuously gets frames from a VideoCapture object
    with a dedicated thread.
    """
    
    def __init__(self, src=0):
    
        self.stream = cv2.VideoCapture(src)
        (self.grabbed, self.frame) = self.stream.read()
        self.stopped = False
        self.hand_detected = False
        
    def start(self):    
        Thread(target=self.get, args=()).start()
        return self

    def get(self):
        global lock
        global distributed_frames
        global frame
        
        ih = is_hand()
        
        start = datetime.now()
        
        
        while not self.stopped:
            if not self.grabbed:
                self.stop()
            else:
              
                (_ , self.frame) = self.stream.read()
                lock.acquire()
                distributed_frames.append(self.frame)
                lock.release()
                
                if(self.hand_detected == False):
                    print("Checking")
                    check_hand = ih.check_hand(self.frame)
                    
                    if(check_hand == True):
                        
                        print("Hand found")
                        self.hand_detected = True
                        distributed_frames.predict_model()
                    
                
                
                           
            
    def stop(self):
        self.stopped = True

In [14]:
def threadVideoGet(source=0):
    """
    Dedicated thread for grabbing video frames with VideoGet object.
    Main thread shows video frames.
    """
    
    #tf.disable_v2_behavior()
    video_getter = VideoGet(source).start()
    cps = CountsPerSec().start()
    time.sleep(1)
    start = datetime.now()
    time_marker = TimeMarker().start()   

In [15]:
distributed_frames = DistributedFrames()
time.sleep(1)
lock = Lock()
#threadVideoGet()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [16]:
frame = cv2.imread("/Users/guilhermeviveiros/Desktop/Part_01/1083/00001.jpg")

#plt.imshow(frame)

ih = is_hand()
ih.check_hand(frame)
#print(is_hand(frame))

False

In [17]:
def save_frames(frames):
    
    for (step,frame) in enumerate(frames):
        
        if step < 10:
            imwrite("/Users/guilhermeviveiros/Desktop/LEI/Fotos/0" + str(step) + ".jpg", frame)
        else:
            imwrite("/Users/guilhermeviveiros/Desktop/LEI/Fotos/" + str(step) + ".jpg", frame)

In [18]:
## import threading
print("[INFO] sampling THREADED frames from webcam...")
vs = cv2.VideoCapture(0)
detect_hand = True


i = 0
frames = np.zeros(shape=(36,75,100,3))

tf.disable_v2_behavior()

with detection_graph.as_default():
    
    with tf.Session(graph=detection_graph) as sess:
            
            model = load_model('/Users/guilhermeviveiros/Desktop/simple_model/best_model_6_supersimple.h5')
            model._make_predict_function() 
            print("Ready to start")
            while True:
                
                ret,frame = vs.read()
                frame = cv2.flip(frame,1)
                frame = imutils.resize(frame, width=600)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                # check to see if the frame should be displayed to our screen
                if detect_hand == True:
                    
                    image_np = frame
                    image_np_expanded = np.expand_dims(image_np,axis=0)
                    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
                    boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
                    scores = detection_graph.get_tensor_by_name('detection_scores:0')
                    classes = detection_graph.get_tensor_by_name('detection_classes:0')
                    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
                      
                    
                        
                        
                    (boxes, scores, classes, num_detections) = sess.run(
                    [boxes, scores, classes, num_detections],
                    feed_dict={image_tensor: image_np_expanded})
                    
                    # Visualization of the results of a detection.
                    vis_util.visualize_boxes_and_labels_on_image_array(
                        image_np,
                        np.squeeze(boxes),
                        np.squeeze(classes).astype(np.int32),
                        np.squeeze(scores),
                        category_index,
                        use_normalized_coordinates=True,
                        max_boxes_to_draw = 2,
                        min_score_thresh=0.90,
                        line_thickness=8)
                    
                    cv2.imshow("Frame", frame)
                    
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        break
                    
                    condition = scores[0][0]*100                    
                        
                    if(condition >= 90):
                        print(condition)
                        detect_hand = False
                    
                    
                            
                            
                #começa a processar o gesto
                if(detect_hand == False):
                    
                    cv2.imshow("Frame", frame)
                    
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        break
                    
                    frame = cv2.resize(frame, (100,75))
                    frames[i] =  frame
                    i +=1;
                    
                    if(i == 36):
                        
                        #save_frames(frames)
                        l = model.predict([[frames]])
                        print(l)
                        l = np.argmax(l)
                        
                        if(l == 0):
                            print("Doing other things")
                        elif(l==1):
                            print("Swipping down")
                            minimize()
                        elif(l==2):
                            print("Swipping left")
                            swipe('right')
                        elif(l==3):
                            print("Zooming") 
                            scale('+')
                        detect_hand = True
                        i=0
                        
                        time.sleep(1)
                        frames = np.zeros(shape=(36,75,100,3))
                        print("Ready to capute the hand")
                        
            # do a bit of cleanup
            #vs.stream.release()
            vs.release()
            cv2.destroyAllWindows()

[INFO] sampling THREADED frames from webcam...
Ready to start
90.90155363082886
[[0. 0. 0. 1.]]
Zooming
Ready to capute the hand
91.95363521575928
[[1. 0. 0. 0.]]
Doing other things
Ready to capute the hand
94.10557150840759
[[0. 0. 1. 0.]]
Swipping left
Ready to capute the hand
97.75825142860413
[[0. 0. 0. 1.]]
Zooming
Ready to capute the hand
97.84116744995117
[[0. 0. 0. 1.]]
Zooming
Ready to capute the hand
92.66498684883118
[[0.000000e+00 0.000000e+00 1.000000e+00 7.571057e-10]]
Swipping left
Ready to capute the hand
99.99263882637024
[[0. 0. 0. 1.]]
Zooming
Ready to capute the hand
96.3716983795166
[[0. 1. 0. 0.]]
Swipping down
Ready to capute the hand
92.23003387451172
[[3.0764449e-10 7.1985878e-06 9.9999285e-01 3.3361897e-10]]
Swipping left
Ready to capute the hand
99.48129653930664
[[0.7787322  0.         0.         0.22126786]]
Doing other things
Ready to capute the hand
99.78511333465576
[[0.0000e+00 0.0000e+00 1.0000e+00 5.9851e-36]]
Swipping left
Ready to capute the hand
97